<a href="https://colab.research.google.com/github/ShaswataJash/kfpcomponent/blob/main/TabularDataPreparationUsingPycaret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is the development workflow for kubeflow pipeline component of the same name as this notebook. Refer https://github.com/ShaswataJash/kfpcomponent

#Install required softwares

In [19]:
!uname -a

Linux 8a647a2648cf 5.4.188+ #1 SMP Sun Apr 24 10:03:06 PDT 2022 x86_64 x86_64 x86_64 GNU/Linux


In [20]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.5 LTS
Release:	18.04
Codename:	bionic


In [4]:
!python --version

Python 3.7.13


In [1]:
!pip install pycaret==2.3.10

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 320 kB 8.6 MB/s 
     |████████████████████████████████| 2.0 MB 47.9 MB/s 
     |████████████████████████████████| 262 kB 54.2 MB/s 
     |████████████████████████████████| 120 kB 53.2 MB/s 
     |████████████████████████████████| 6.8 MB 23.3 MB/s 
     |████████████████████████████████| 167 kB 77.6 MB/s 
     |████████████████████████████████| 56 kB 4.3 MB/s 
     |████████████████████████████████| 1.3 MB 37.4 MB/s 
     |████████████████████████████████| 88 kB 8.5 MB/s 
     |████████████████████████████████| 17.8 MB 33.5 MB/s 
     |████████████████████████████████| 1.7 MB 38.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.3 MB 50.5 MB/s 
     |████████████████████████████████| 

In [2]:
#Ref: https://github.com/pycaret/pycaret/issues/2490
!pip install Jinja2==3.1.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 133 kB 6.3 MB/s 
  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
!apt-get install ca-certificates fuse tzdata rclone && \
  echo "user_allow_other" >> /etc/fuse.conf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fuse is already the newest version (2.9.7-1ubuntu1).
ca-certificates is already the newest version (20210119~18.04.2).
tzdata is already the newest version (2022a-0ubuntu0.18.04).
tzdata set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  rclone
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 3,186 kB of archives.
After this operation, 12.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 rclone amd64 1.36-3 [3,186 kB]
Fetched 3,186 kB in 2s (2,087 kB/s)
Selecting previously unselected package rclone.
(Reading database ... 155632 files and directories currently installed.)
Preparing to unpack .../rclone_1.36-3_amd64.deb ...
Unpacking rclone (1.36-3) ...
Se

#Develop source code files

In [4]:
%%writefile data_preparation.py
#!/usr/bin/env python3
import argparse

# Defining and parsing the command-line arguments
parser = argparse.ArgumentParser(description='kubeflow pipeline component to read csv file and prepare the data')
parser.add_argument('--input-datasource-directory-mountable', default=False, action="store_true", help='whether input csv file is present in mountable remote location')
parser.add_argument('--input-datasource-directory-to-be-mounted', type=str, help='if input-datasource-directory-mountable=True, name of the mountable directory (e.g. bucket name for s3)')
parser.add_argument('--input-datasource-file-name', type=str, help='name of the csv file including file extension (if any)')
parser.add_argument('--additional-options-csv-parsing', type=str, default= '{}', help='json formatted key-value pairs of strings which will be passed to pandas.read_csv()')
parser.add_argument('--type-of-data-analysis-task', choices=['classification', 'regression', 'clustering', 'anomaly_detection'])
parser.add_argument('--target-variable-name', type=str, help='for classification and regression, specify the column name holding target variable')
parser.add_argument('--target-emptyindicator', type=str, default='', help='if target variable column holds null or na, those rows will be dropped. Sometime empty can be indicated by other representative string like - or *** etc')
parser.add_argument('--data-preparations-options', type=str, default= '{}', help='json formatted key-value pairs of strings which will be passed to pycaret setup() function')
parser.add_argument('--additional-options-csv-writing', type=str, default= '{}', help='json formatted key-value pairs of strings which will be passed to pandas.to_csv()')
parser.add_argument('--output-datasource-directory-mountable', default=False, action="store_true", help='whether output csv file will be written in mountable remote location')
parser.add_argument('--output-datasource-containing-directory', type=str, help='name of the directory (e.g. bucket name for s3) where csv file will be written')
parser.add_argument('--output-datasource-file-name', type=str, help='filename of the prepared data')
args = parser.parse_args()

import tempfile
local_datastore_read_dir = tempfile.mkdtemp(prefix="my_local_read-")
print('local_datastore_read_dir:',local_datastore_read_dir)

local_datastore_write_dir = tempfile.mkdtemp(prefix="my_local_write-")
print('local_datastore_write_dir:',local_datastore_write_dir)

#input file handling
import subprocess
import os
import sys
if args.input_datasource_directory_mountable:
    input_data_read_cmd = "rclone -v mount remoteread:" + args.input_datasource_directory_to_be_mounted + ' ' + local_datastore_read_dir + ' --daemon'
else:
    input_data_read_cmd = "rclone -v copy remoteread:" + args.input_datasource_file_name + ' ' + local_datastore_read_dir
input_data_read_call = subprocess.run(input_data_read_cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
print(input_data_read_call.stdout)
if input_data_read_call.returncode != 0:
    print("Error in rclone, errorcode=", input_data_read_call.returncode)
    sys.stdout.flush()
    sys.exit("Forceful exit as rclone returned error in context of reading")

#output file handling
if args.output_datasource_directory_mountable:
    output_data_write_cmd = "rclone -v mount remotewrite:" + args.output_datasource_containing_directory + ' ' + local_datastore_write_dir + ' --daemon'
    output_data_write_call = subprocess.run(output_data_write_cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    print(output_data_write_call.stdout)
    if output_data_write_call.returncode != 0:
        print("Error in rclone, errorcode=", output_data_write_call.returncode)
        sys.stdout.flush()
        sys.exit("Forceful exit as rclone returned error in context of mounted writing")

#handling input csv file reading
import pandas
import json

try:
    parse_config = json.loads(args.additional_options_csv_parsing)
    print('parse_config = (', type(parse_config), ')', parse_config)
    parse_config['filepath_or_buffer'] = os.path.join(local_datastore_read_dir,args.input_datasource_file_name)
    my_data = pandas.read_csv(**parse_config)
    print(my_data)
    
except BaseException as err:
    print("Error=", err, ' ', type(err))
    sys.stdout.flush()
    sys.exit("Forceful exit as exception encountered while parsing input csv file")

#handling data preprocessing
import pycaret
try:
    if os.path.exists("logs.log"):
        os.remove("logs.log") #removing any content from log which pycaret will internally use for its own logging
    print('pycaret version = ', pycaret.utils.version())
    setup_config = json.loads(args.data_preparations_options)
    print('setup_config = (', type(setup_config), ')', setup_config)
    if args.type_of_data_analysis_task == 'classification':
        import pycaret.classification
        setup_fn = pycaret.classification.setup
        get_config_fn = pycaret.classification.get_config
        setup_config['target'] = args.target_variable_name
        
    elif args.type_of_data_analysis_task == 'regression':
        import pycaret.regression
        setup_fn = pycaret.regression.setup
        get_config_fn = pycaret.regression.get_config
        setup_config['target'] = args.target_variable_name

    elif args.type_of_data_analysis_task == 'clustering':
        import pycaret.clustering
        setup_fn = pycaret.clustering.setup
        get_config_fn = pycaret.clustering.get_config

    elif args.type_of_data_analysis_task == 'anomaly':
        import pycaret.anomaly
        setup_fn = pycaret.anomaly.setup
        get_config_fn = pycaret.anomaly.get_config
        
    #as part of pycaret's data cleaning the rows with target column = nan are not being cleaned up. Thus, cleaning those rows explicitely
    if len(args.target_emptyindicator) > 0:
        #ref: https://stackoverflow.com/questions/49291740/delete-rows-if-there-are-null-values-in-a-specific-column-in-pandas-dataframe
        import numpy as np
        my_data[args.target_variable_name] = my_data[args.target_variable_name].replace(args.target_emptyindicator, np.nan)
        my_data = my_data.dropna(axis=0, subset=[args.target_variable_name])

    setup_config['data'] = my_data
    setup_config['log_experiment'] = False
    setup_config['data_split_shuffle'] = False
    setup_config['html'] = False
    setup_config['silent'] = True
    setup_fn(**setup_config)
    #ref: https://www.kdnuggets.com/2020/11/5-things-doing-wrong-pycaret.html
    X_transformed = get_config_fn('X')
    my_transformed_data = X_transformed
    if args.type_of_data_analysis_task == 'classification' or args.type_of_data_analysis_task == 'regression':
        y_transformed = get_config_fn('y')
        my_transformed_data = X_transformed.merge(y_transformed,left_index=True, right_index=True)
    print(my_transformed_data)

    pycaret.utils.get_system_logs() #this will print the pycaret's own log into console
    
except BaseException as err:
    pycaret.utils.get_system_logs()
    print("Error=", err, ' ', type(err))
    sys.stdout.flush()
    sys.exit("Forceful exit as exception encountered while transforming input dataframe")

#handling output csv file writing
try:
    to_csv_config = json.loads(args.additional_options_csv_writing)
    print('to_csv_config = (', type(to_csv_config), ')', to_csv_config)
    to_csv_config['path_or_buf'] = os.path.join(local_datastore_write_dir,args.output_datasource_file_name)
    my_transformed_data.to_csv(**to_csv_config)
except BaseException as err:
    print("Error=", err, ' ', type(err))
    sys.stdout.flush()
    sys.exit("Forceful exit as exception encountered while trying to write prepared data")

if not args.output_datasource_directory_mountable:
    output_data_write_cmd = "rclone -v copy " + os.path.join(local_datastore_write_dir,args.output_datasource_file_name) + " remotewrite:" + args.output_datasource_containing_directory + '/'
    output_data_write_call = subprocess.run(output_data_write_cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    print(output_data_write_call.stdout)
    if output_data_write_call.returncode != 0:
        print("Error in rclone, errorcode=", output_data_write_call.returncode)
        sys.stdout.flush()
        sys.exit("Forceful exit as rclone returned error in context of writing final csv file (copy mode)")

Writing data_preparation.py


In [32]:
%%writefile Dockerfile
FROM python:3.7.13-slim
RUN python3 -m pip install pycaret==2.3.10
RUN python3 -m pip install Jinja2==3.1.2
RUN apt update
RUN apt-get -y install ca-certificates tzdata fuse rclone && echo "user_allow_other" >> /etc/fuse.conf
#additional installation for pycaret in python-slim
RUN apt-get -y install libgomp1
COPY src/data_preparation.py /tmp
COPY tests/test_validation.py /tmp
COPY run_tests.sh /tmp
RUN chmod 544 /tmp/run_tests.sh

Overwriting Dockerfile


In [5]:
%%writefile run_tests.sh
#!/bin/bash

export RCLONE_CONFIG_REMOTEREAD_TYPE='http'
export RCLONE_CONFIG_REMOTEREAD_URL='https://raw.githubusercontent.com/pycaret/datasets/main/data/common/'

export RCLONE_CONFIG_REMOTEWRITE_TYPE='local'
export RCLONE_CONFIG_REMOTEWRITE_NOUNC='true'

mkdir /tmp/my_local_dir_for_test

#Test: csv reading source from http, rclone read in copy, rclone write in mount mode
#python /tmp/data_preparation.py --input-datasource-file-name 'CTG.csv' --additional-options-csv-parsing '{"sep":"," , "header":0}' \
#    --type-of-data-analysis-task 'classification' --target-variable-name 'NSP' \
#    --data-preparations-options '{"ignore_low_variance":true, "remove_outliers":true, "remove_multicollinearity":true, "multicollinearity_threshold":0.7}' \
#    --output-datasource-directory-mountable --output-datasource-containing-directory '/tmp/my_local_dir_for_test' --output-datasource-file-name 'CTG_data-prep.csv'

#https://registry.opendata.aws/humor-detection/
export RCLONE_CONFIG_REMOTEREAD_TYPE='s3'
export RCLONE_CONFIG_REMOTEREAD_PROVIDER='AWS'
export RCLONE_CONFIG_REMOTEREAD_ACCESS_KEY_ID=''
export RCLONE_CONFIG_REMOTEREAD_SECRET_ACCESS_KEY=''
export RCLONE_CONFIG_REMOTEREAD_REGION='us-west-2' 
export RCLONE_CONFIG_REMOTEREAD_ENDPOINT= ''

#Test: csv reading source from s3(AWS provider), rclone read in mount, rclone write in copy mode
python /tmp/data_preparation.py --input_datasource_directory_mountable --input_datasource_directory_to_be_mounted 'humor-detection-pds/' --input-datasource-file-name 'Non-humours-biased.csv' \
    --type-of-data-analysis-task 'classification' --target-variable-name 'label' \
    --data-preparations-options '{"ignore_features":{"image_url"}}' \
    --output-datasource-containing-directory '/tmp/my_local_dir_for_test' --output-datasource-file-name 'Non-humours-biased_data-prep.csv'

python /tmp/test_validation.py

Writing run_tests.sh


In [4]:
%%writefile test_validation.py
#!/usr/bin/env python3
import pandas
df = pandas.read_csv(filepath_or_buffer = '/tmp/my_local_dir_for_test/CTG_data-prep.csv')
assert len(df.index) == 2126 #original data had 2129 rows, amongst that 3 rows have no target
assert df.isnull().sum().sum() == 0 #pycaret will remove all missing values
print ('test-validation done successfully')

Writing test_validation.py


Refer: https://github.com/RealOrangeOne/docker-rclone-mount/blob/master/docker-compose.yml

If we have to use mount feature of rclone, it needs to have fuse support in underneath linux kernel. For that we are adding SYS_ADMIN in capability. But note without using mount feature also, we can do testing. in that case, rclone will use only copy feature.

In [5]:
%%writefile docker-compose.test.yml
services:
  sut:
    build: .
    command: /tmp/run_tests.sh
    cap_add:
      - SYS_ADMIN
    security_opt:
      - apparmor:unconfined
    devices:
      - "/dev/fuse:/dev/fuse"

Writing docker-compose.test.yml


https://github.com/kubeflow/pipelines/blob/sdk/release-1.8/sdk/python/kfp/dsl/types.py

In [6]:
%%writefile component.yaml
name: TabularDataPreparationUsingPycaret
description: |
    Prepare tabular data (csv file) using pycaret library. (For pycaret's data pre-processing capabilities, refer https://pycaret.gitbook.io/docs/get-started/preprocessing)
    pycaret internally uses pandas dataframe to read and write csv file. 
    Input and processed csv file are stored in rclone compatible storage. Both mount and copy mode are supported. (refer: https://rclone.org/)
    rclone configurations have to be shared through environment variables (refer: https://rclone.org/docs/#environment-variables). 
    Thus, before using this component in kubeflow pipeline, those environment variables have to be set from pipeline.
    Create rclone read and write configuration file name as 'REMOTEREAD' and 'REMOTEWRITE'. Because the same are used within code.
    So convention for creating any environment variables related to rclone should start either with 'RCLONE_CONFIG_REMOTEREAD' or 'RCLONE_CONFIG_REMOTEWRITE' 

inputs:
- {name: input-datasource-directory-mountable, optional, description: 'whether input csv file is present in mountable remote location'}
- {name: input-datasource-directory-to-be-mounted, type: String, description: 'if input-datasource-directory-mountable=True, name of the mountable directory (e.g. bucket name for s3)'}
- {name: input-datasource-file-name, type: String, description: 'name of the csv file including file extension (if any)'}
- {name: additional-options-csv-parsing, type: String, default= '{}', description: 'json formatted key-value pairs of strings which will be passed to pandas.read_csv()'}
- {name: type-of-data-analysis-task, type: List, description: 'choice amongst classification, regression, clustering, anomaly_detection'}
- {name: target-variable-name, type: String, description: 'for classification and regression, specify the column name holding target variable'}
- {name: target-emptyindicator, type: String, default='', description: 'if target variable column holds null or na, those rows will be dropped. Sometime empty can be indicated by other representative string like - or *** etc'}
- {name: data-preparations-options, type: String, default= '{}', description: 'json formatted key-value pairs of strings which will be passed to pycaret setup() function'}
- {name: additional-options-csv-writing, type: String, default= '{}', description: 'json formatted key-value pairs of strings which will be passed to pandas.to_csv()'}
- {name: output-datasource-directory-mountable, optional, description: 'whether output csv file will be written in mountable remote location'}
- {name: output-datasource-containing-directory, type: String, description: 'name of the directory (e.g. bucket name for s3) where csv file will be written'}
- {name: output-datasource-file-name, type: String, description: 'filename of the prepared data'}

implementation:
  container:
    image: hub.docker.com/shasjash/kfpcomponents/TabularDataPreparationUsingPycaret_devlatest
    # command is a list of strings (command-line arguments). 
    # The YAML language has two syntaxes for lists and you can use either of them. 
    # Here we use the "flow syntax" - comma-separated strings inside square brackets.
    command: [
      python3, 
      # Path of the program inside the container
      /tmp/data_preparation.py,
      --input-datasource-directory-mountable,
      {inputValue: input-datasource-directory-mountable},
      --input-datasource-directory-to-be-mounted, 
      {inputValue: input-datasource-directory-to-be-mounted},
      --input-datasource-file-name, 
      {inputValue: input-datasource-file-name},
      --additional-options-csv-parsing, 
      {inputValue: additional-options-csv-parsing},
      --type-of-data-analysis-task, 
      {inputValue: type-of-data-analysis-task},
      --target-variable-name, 
      {inputValue: target-variable-name},
      --target-emptyindicator, 
      {inputValue: target-emptyindicator},
      --data-preparations-options, 
      {inputValue:data-preparations-options},
      --additional-options-csv-writing, 
      {inputValue: additional-options-csv-writing},
      --output-datasource-directory-mountable, 
      {inputValue: output-datasource-directory-mountable},
      --output-datasource-containing-directory, 
      {inputValue: output-datasource-containing-directory},
      --output-datasource-file-name, 
      {inputValue: output-datasource-file-name},
    ]

Writing component.yaml


#Software testing

Now let us simulate testing what will be done by docker-hub infrastructure as part of auto-testing by using docker-compose.test.yml present in github source repository.

In [6]:
!rm -rf /tmp/my_local_dir_for_test
!chmod 544 run_tests.sh
!cp data_preparation.py /tmp
!cp test_validation.py /tmp
!./run_tests.sh

cp: cannot stat 'test_validation.py': No such file or directory
./run_tests.sh: line 23: export: `': not a valid identifier
usage: data_preparation.py [-h] [--input-datasource-directory-mountable]
                           [--input-datasource-directory-to-be-mounted INPUT_DATASOURCE_DIRECTORY_TO_BE_MOUNTED]
                           [--input-datasource-file-name INPUT_DATASOURCE_FILE_NAME]
                           [--additional-options-csv-parsing ADDITIONAL_OPTIONS_CSV_PARSING]
                           [--type-of-data-analysis-task {classification,regression,clustering,anomaly_detection}]
                           [--target-variable-name TARGET_VARIABLE_NAME]
                           [--target-emptyindicator TARGET_EMPTYINDICATOR]
                           [--data-preparations-options DATA_PREPARATIONS_OPTIONS]
                           [--additional-options-csv-writing ADDITIONAL_OPTIONS_CSV_WRITING]
                           [--output-datasource-directory-mountable]
    

Before commiting code to github, install github client (gh) by following instruction mentioned in https://github.com/cli/cli/blob/trunk/docs/install_linux.md (Choose Debian, Ubuntu Linux way of installation) 

Use the colab's 'Terminal' icon present in left vertical pane to open linux terminal to type commands. Once 'gh' is installed, type **$gh auth login** (refer https://docs.github.com/en/get-started/getting-started-with-git/caching-your-github-credentials-in-git) to follow onscreen prompts. For colab, use **Paste an authentication token** option. Personal tokens can be generated in https://github.com/settings/tokens

You can use Shift+Ctrl+v shortcut to paste any string in colab console

#Push the code to github

In [33]:
!pwd

/content


In [34]:
!rm -Rf kfpcomponent

In [35]:
!git clone https://github.com/ShaswataJash/kfpcomponent.git

Cloning into 'kfpcomponent'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 102 (delta 44), reused 78 (delta 29), pack-reused 0
Receiving objects: 100% (102/102), 40.41 KiB | 713.00 KiB/s, done.
Resolving deltas: 100% (44/44), done.


Follow directory structure according to https://www.kubeflow.org/docs/components/pipelines/sdk/component-development/#organizing-the-component-files

In [36]:
!mkdir kfpcomponent/TabularDataPreparationUsingPycaret
!mkdir kfpcomponent/TabularDataPreparationUsingPycaret/src
!mkdir kfpcomponent/TabularDataPreparationUsingPycaret/tests

mkdir: cannot create directory ‘kfpcomponent/TabularDataPreparationUsingPycaret’: File exists
mkdir: cannot create directory ‘kfpcomponent/TabularDataPreparationUsingPycaret/src’: File exists
mkdir: cannot create directory ‘kfpcomponent/TabularDataPreparationUsingPycaret/tests’: File exists


In [37]:
#it will ensure file is coped in git repo only if file content is changed by checking checksum of file content
!rsync -c data_preparation.py kfpcomponent/TabularDataPreparationUsingPycaret/src
!rsync -c component.yaml kfpcomponent/TabularDataPreparationUsingPycaret/component.yaml
!rsync -c test_validation.py kfpcomponent/TabularDataPreparationUsingPycaret/tests
!rsync -c Dockerfile kfpcomponent/TabularDataPreparationUsingPycaret/
!rsync -c run_tests.sh kfpcomponent/TabularDataPreparationUsingPycaret/
!rsync -c docker-compose.test.yml kfpcomponent/TabularDataPreparationUsingPycaret/

In [38]:
%cd kfpcomponent

/content/kfpcomponent


In [39]:
!git add -A

In [40]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	modified:   TabularDataPreparationUsingPycaret/Dockerfile



For git-user who has set their email visibility as private, git provides alternate email address to use in web-based Git operations, e.g., edits and merges. The alias email can be viewed in https://github.com/settings/emails

In [41]:
!git config --global user.email "29448766+ShaswataJash@users.noreply.github.com"

In [42]:
!git commit -a -m "changed docker to install libgomp1"

[main 42a9e42] changed docker to install libgomp1
 1 file changed, 2 insertions(+), 1 deletion(-)


In [43]:
!git push origin main

Counting objects: 4, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 465 bytes | 465.00 KiB/s, done.
Total 4 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/ShaswataJash/kfpcomponent.git
   31d731c..42a9e42  main -> main


In [44]:
%cd ..

/content
